In [99]:
import sympy
import numpy as np

In [100]:
A = np.array([[2, 4, -1, 5, 2],
          [-4, -5, 3, -8, 1],
          [2, -5, -4, 1, 8],
          [-6, 0, 7, -3, 1]],dtype=np.float64)

In [101]:
count_el_diag = min(A.shape)

In [102]:
L = np.zeros(A.shape, dtype=np.float64)
U = A.copy()

In [103]:
H = A.shape[0]
W = A.shape[1]
eps = 1e-12


for i in range(count_el_diag):
    L[i, i] = 1.0

    if abs(U[i, i]) < eps:
        max_row = np.argmax(np.abs(U[i:, i])) + i
        if abs(U[max_row, i]) < eps:
            continue
        if max_row != i:
            U[[i, max_row], :] = U[[max_row, i], :]
            if i > 0:
                L[[i, max_row], :i] = L[[max_row, i], :i]

    for j in range(i+1, H):
        mult = U[j, i] / U[i, i]
        L[j, i] = mult
        U[j, i:] = U[j, i:] - mult * U[i, i:]
        U[j, i] = 0.0
        U[j][np.abs(U[j]) < eps] = 0.0


print(L)
print(U)

[[ 1.  0.  0.  0.  0.]
 [-2.  1.  0.  0.  0.]
 [ 1. -3.  1.  0.  0.]
 [-3.  4.  0.  1.  0.]]
[[  2.   4.  -1.   5.   2.]
 [  0.   3.   1.   2.   5.]
 [  0.   0.   0.   2.  21.]
 [  0.   0.   0.   4. -13.]]
